In [ ]:
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt

import shapely as sp
import shapely.geometry

In [ ]:
import polymetric as pm
reload(pm)

In [ ]:
import polymetric.exporters.gds
import polymetric.exporters.dxf

In [ ]:
def plot_polys(polys, cycle_colours=False, **kw):
    single_color = None
    for i, outer_poly in enumerate(polys):
        if isinstance(outer_poly, (sp.geometry.MultiPolygon)):
            inner_poly = outer_poly.geoms
        else:
            inner_poly = [outer_poly]
        
        for poly in inner_poly:
            xs, ys = poly.exterior.xy
            plot_params = dict(kw)

            if cycle_colours:
                plot_params["color"] = "C%d" % (i % 10)
            elif single_color is not None:
                plot_params["color"] = single_color

            l = plt.plot(xs, ys, **plot_params)
            if single_color is None:
                single_color = l[0].get_color()
                plot_params["color"] = single_color

            # plot interiors with a different line style
            plot_params["linestyle"] = "--"
            for interior in poly.interiors:
                xs, ys = interior.xy
                plt.plot(xs, ys, **plot_params)
            
def plot_shapes(shapes, *more_shapes, **kw):
    if not isinstance(shapes, (list, tuple)):
        shapes = [shapes]
        
    shapes += more_shapes
    for s in shapes:
        plot_polys(s.polygonize(), **kw)
            
def show_polys():
    plt.gca().set_aspect('equal')
    plt.show()
    
def inspect_shapes(shapes, *more_shapes, **kw):
    plt.figure()
    plot_shapes(shapes, *more_shapes, **kw)
    show_polys()

## Parameters

In [ ]:
hole_b = 0.750/2.0
hole_a = 0.270/2.0

slit_d = 0.045
beam_w = 0.900
beam_l = 10.0

holes_n = 11

defect_d = 0.400
lattice_d = 0.450

In [ ]:
hole = pm.Ellipse(a=hole_a, b=hole_b, x=defect_d/2)
inspect_shapes(hole)

In [ ]:
hole_sweep = pm.ParametricSweep(
    constructor=hole.clone,
    sweep_over=np.arange(holes_n),
    sweep_params = {
        'x': lambda n: defect_d/2 + n*lattice_d,
#         'b': lambda n: hole_b if n > 2 else (0.7 + 0.1*n)*hole_b,
#         'a': lambda n: hole_a if n > 2 else (0.7 + 0.1*n)*hole_a,
    }
)

inspect_shapes(hole_sweep)

In [ ]:
hole_sweep_mirror = pm.Scaled(hole_sweep, scales=(-1.0, 1.0))
inspect_shapes(hole_sweep, hole_sweep_mirror)

In [ ]:
all_holes = pm.Combined([hole_sweep, hole_sweep_mirror])
inspect_shapes(all_holes)

In [ ]:
full_beam = pm.Rectangle(x=0.0, y=0.0, w=beam_l, h=beam_w)
inspect_shapes(full_beam)

In [ ]:
minus_holes = pm.Difference([full_beam, all_holes])
inspect_shapes(minus_holes)

In [ ]:
minus_holes.has_interiors()

In [ ]:
minus_slit = pm.Difference([minus_holes, pm.Rectangle(x=0.0, y=0.0, w=beam_l, h=slit_d)])
inspect_shapes(minus_slit)

In [ ]:
minus_slit.has_interiors()

In [ ]:
final_polys = minus_slit.apply(pm.Expanded).polygonize()

# GDSpy stuff starts here

In [ ]:
shape_to_save = minus_slit

In [ ]:
import gdspy
reload(gdspy)

MAX_VERTICES = 8000

In [ ]:
poly_cell = gdspy.Cell('johnbeam')

In [ ]:
# for spoly in final_polys:
#     if len(spoly.interiors) > 0:
#         print("WARNING: polygon contains interiors, which is not supported by the GDS file format. Ignoring...")
        
#     vertices = list(spoly.exterior.coords)
#     n_vertices = len(vertices)
#     print("Number of vertices: %d" % n_vertices)
#     if n_vertices > MAX_VERTICES:
#         raise ValueError("Polygon contains more that %d vertices" % MAX_VERTICES)
#     gds_poly = gdspy.Polygon(vertices, 1)
    
#     poly_cell.add(gds_poly)

# poly_cell.flatten(single_datatype=1000)

In [ ]:
if shape_to_save.has_interiors():
    print("WARNING: polygon contains interiors, which is not supported by the GDS file format. Ignoring the interiors.")
    
vertex_lists = shape_to_save.get_exterior_vertex_lists()

for vl in vertex_lists:
    if len(vl) > MAX_VERTICES:
        raise ValueError("Polygon contains more that %d vertices: %d" % (MAX_VERTICES, len(vl)))
    gds_poly = gdspy.Polygon(vl[:-1], 1)
    poly_cell.add(gds_poly)
    
poly_cell.flatten(single_datatype=1000)

In [ ]:


gdspy.write_gds("johnbeam.gds", unit=1e-6, precision=1e-9)


# DXF stuff starts here (dxfwrite)

In [ ]:
from dxfwrite import DXFEngine as dxf

drawing = dxf.drawing("johnbeam.dxf")

vertex_lists = shape_to_save.get_exterior_vertex_lists()

for vl in vertex_lists:
    polyline = dxf.polyline(thickness=0, startwidth=0, endwidth=0)
    # shapely always duplicates the first vertex as the last vertex, get rid of that
    polyline.add_vertices(vl[:-1]) 
    # close polyline explicitly
    polyline.close()
    drawing.add(polyline)

drawing.save()


# DXF stuff with ezdxf

In [ ]:
import ezdxf

dwg = ezdxf.new("R2010")
msp = dwg.modelspace()

vertex_lists = shape_to_save.get_exterior_vertex_lists()

for vl in vertex_lists:
    # shapely always duplicates the first vertex as the last vertex, get rid of that
    polyline = msp.add_lwpolyline(vl[:-1])
    polyline.closed = True
    
dwg.saveas("jb_ezdxf.dxf")


# Saving using polymetric functions

In [ ]:
pm.exporters.gds.save(shape_to_save, "pmsave.gds")

In [ ]:
pm.exporters.dxf.save(shape_to_save, "pmsave.dxf")